# Interfaces Gráficas - Avanzado

## Threads vs. QThread

En PyQt se puede usar tanto **`threading.Thread`** como **`QThread`**

### Comparación básica entre Thread y QThread

```python
import threading
import time

def tarea():
    for i in range(3):
        print(f"Thread estándar ejecutando... {i}")
        time.sleep(1)

hilo = threading.Thread(target=tarea)
hilo.start()
hilo.join()
```

---

```python
from PyQt5.QtCore import QThread

class HiloPyQt(QThread):
    def run(self):
        for i in range(3):
            print(f"QThread ejecutando... {i}")
            self.msleep(1000)  # versión segura de sleep

hilo_qt = HiloPyQt()
hilo_qt.start()
hilo_qt.wait()
```

---

### Médotos Equivalentes

- En `threading.Thread`, se usa `.is_alive()`
- En `QThread`, se usa `.isRunning()`


| Concepto / Método                            | `threading.Thread` (Estándar de Python) | `QThread` (PyQt) | Descripción / Equivalencia |
|----------------------------------------------|------------------------------------------|------------------|-----------------------------|
| **Clase base**                               | `threading.Thread`                      | `PyQt5.QtCore.QThread` | Ambas representan hilos de ejecución. |
| **Inicialización**                           | `super().__init__()` o `Thread(target=...)` | `super().__init__()` | Se inicializan de forma similar. |
| **Inicio del hilo**                          | `.start()`                              | `.start()`       | Ejecuta el método `run()` en un hilo nuevo. |
| **Método principal de ejecución**            | `.run()`                                | `.run()`         | Se redefine para definir la lógica del hilo. |
| **Comprobación si el hilo está activo**      | `.is_alive()`                           | `.isRunning()`   | Verifica si el hilo aún se está ejecutando. |
| **Pausa (dormir)**                           | `time.sleep(segundos)`                  | `self.msleep(ms)` o `QThread.sleep(segundos)` | En `QThread`, se usan métodos propios para dormir. |
| **Finalizar hilo**                           | No hay método seguro (solo con flags)   | `.terminate()`  | `terminate()` detiene bruscamente el hilo, pero puede ser inseguro. |
| **Sincronización / Locks**                   | `threading.Lock()`                      | `QMutex()`       | Ambos evitan condiciones de carrera. |
| **Comunicación entre hilos**                 | Colas (`queue.Queue`) o variables compartidas | Señales (`pyqtSignal`) | En PyQt, las señales son más seguras y evitan errores de concurrencia. |
| **Interacción con GUI**                      |  No seguro modificar GUI directamente |  Seguro mediante señales | Solo las señales garantizan seguridad al modificar la GUI. |
| **Método para esperar la finalización**      | `.join()`                               | `.wait()`        | Ambos bloquean hasta que el hilo termina. |
| **Evento de terminación controlada**         | `threading.Event()`                     | No aplicable directamente | En PyQt se usa un flag manual o señales para detener un hilo. |
| **Métodos de control del tiempo**            | `time.sleep()`, `threading.Timer()`     | `QThread.sleep()`, `QTimer` | PyQt usa su propio temporizador y control de tiempo. |
| **Comunicación periódica**                   | `threading.Timer` (una vez)             | `QTimer` (periódico) | `QTimer` permite repeticiones periódicas fácilmente. |
| **Conexión de eventos o señales**            | No disponible nativamente               | `.connect()` de señales | `QThread` permite enviar señales a otros objetos PyQt. |
| **Compatibilidad con Event Loop de PyQt**    |  No compatible                        |  Totalmente compatible | QThread integra el hilo con el ciclo de eventos Qt. |

## QTimer: Concurrencia Periódica

`QTimer` ejecuta una función cada cierto intervalo.  
Es ideal para tareas **repetitivas**, sin necesidad de crear hilos.

### Actualización periódica

```python
from PyQt5.QtCore import QTimer
from PyQt5.QtWidgets import QApplication, QLabel, QWidget, QVBoxLayout
import sys, datetime

class Reloj(QWidget):
    def __init__(self):
        super().__init__()
        self.label = QLabel()
        layout = QVBoxLayout()
        layout.addWidget(self.label)
        self.setLayout(layout)

        self.timer = QTimer()
        self.timer.setInterval(1000)  # 1 segundo
        self.timer.timeout.connect(self.actualizar_hora)
        self.timer.start()

    def actualizar_hora(self):
        self.label.setText(datetime.datetime.now().strftime("%H:%M:%S"))

app = QApplication(sys.argv)
w = Reloj()
w.show()
app.exec_()
```


<div class="alert alert-block alert-info">
  <b>Tarea:</b> Hacer un reloj que muestre la hora usando Threads y otro usando QThreads o QTimer en "clock.py"
</div>

1. Modifique la función `initialize` la cual debe incluir los estilos colocados en `custom_style`

2. Haga uso de `QTimer` para incluirlo en un QLabel y mostrar la hora actual. 

## Señales y QThreads

En PyQt **no se debe modificar la GUI directamente** desde un hilo secundario.  
Para ello, se usan **señales (`pyqtSignal`)**, que garantizan una comunicación segura entre hilos.

### Comunicación entre hilo y GUI

```python
from PyQt5.QtCore import QThread, pyqtSignal
from PyQt5.QtWidgets import QApplication, QLabel, QWidget, QVBoxLayout
import sys, time

class Worker(QThread):
    progreso = pyqtSignal(int)

    def run(self):
        for i in range(1, 6):
            time.sleep(1)
            self.progreso.emit(i * 20)

class Ventana(QWidget):
    def __init__(self):
        super().__init__()
        self.label = QLabel("Progreso: 0%")
        layout = QVBoxLayout()
        layout.addWidget(self.label)
        self.setLayout(layout)

        self.worker = Worker()
        self.worker.progreso.connect(self.actualizar_label)
        self.worker.start()

    def actualizar_label(self, valor):
        self.label.setText(f"Progreso: {valor}%")

app = QApplication(sys.argv)
v = Ventana()
v.show()
app.exec_()
```

<div class="alert alert-block alert-info">
  <b>Tarea:</b> Crear un salvapantallas de DVD
</div>

1. **`dvd.py`** Crea una clase `DVD` que herede de `QSvgWidget` muestre el logo ubicado en `resources/DVD_logo.svg`  
   - Debe tener métodos para cambiar su posición y color.


2. **`tv.py`** Cree un `QThread` llamado `MovimientoThread` que tenga los siguientes requisitos:
   - Se instancia con los argumentos de ancho y alto de la pantalla, es decir: `def __init__(self, ancho: int, alto: int, velocidad: int=1, dimensiones_logo: tuple[int,int]=(120, 80)):`.
   - Emite una `tuple` y un objeto `QColor`.
   - Cree los siguientes métodos:
      - `run`: Incluye el loop principal donde si está corriendo el `QThread` evalua si hay colisión y sino sigue moviendo el objeto.
      - `collisions`: Esta función detecta si hubo una colisión de el logo DVD de dimensiones dadas al instanciar `MovimientoThread`
      - `stop`: este método se encarga de detener el `QThread`
   - Emita una **señal** cada vez que haya un rebote, enviando la **nueva posición y color**.
3. **`tv.py`** Conecta las señales del hilo con métodos de la clase principal (`Pantalla`) para actualizar la GUI.
   - Inicie el _thread_ de `MovimientoThread`, conecte la señal con el método `mover_logo`
   - Ubique, e instancie DVDLogo, y Clock. Donde DVD logo recibe `DVD_PATH` como parámetro dado.

## Reproducción de Sonido

Dos formas principales:

- `QMediaPlayer`: para archivos **.mp3**
- `QSoundEffect`: para archivos **.wav**

### QSoundEffect

```python
from PyQt5.QtMultimedia import QSoundEffect
from PyQt5.QtCore import QUrl
import time

sonido = QSoundEffect()
sonido.setSource(QUrl.fromLocalFile("/ruta/sonido.wav"))
sonido.play()
time.sleep(2)
```

<div class="alert alert-block alert-info">
  <b>Tarea:</b> Ahora, en cada colisión, reproduzca el sonido: `resources/sus.mp3`
</div>

## Estructura de Ventana Principal (QMainWindow)

`QMainWindow` ofrece la estructura clásica de una aplicación:  
menús, barras de herramientas, barra de estado y área central.

### Ejemplo básico

```python
from PyQt5.QtWidgets import QApplication, QMainWindow, QTextEdit
import sys

class VentanaPrincipal(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Ejemplo QMainWindow")
        self.setCentralWidget(QTextEdit())
        self.statusBar().showMessage("Listo")

app = QApplication(sys.argv)
v = VentanaPrincipal()
v.show()
app.exec_()
```

## Control de Acceso con QMutex (Locks)

Cuando múltiples hilos acceden a un mismo recurso, se puede usar un **`QMutex`** para evitar conflictos.

### Uso de QMutex para proteger una sección crítica

```python
from PyQt5.QtCore import QThread, QMutex
import time

mutex = QMutex()
contador = 0

class Sumador(QThread):
    def run(self):
        global contador
        for _ in range(1000):
            mutex.lock()
            contador += 1
            mutex.unlock()

hilos = [Sumador() for _ in range(5)]
for h in hilos: h.start()
for h in hilos: h.wait()

print("Contador final:", contador)
```
---

## Integración de Networking con Interfaces Gráficas

En una aplicación cliente-servidor con GUI:

- El **cliente** maneja la interfaz y levanta un hilo que escucha al servidor.
- El **servidor** no necesita GUI, solo maneja las solicitudes.


```python
from PyQt5.QtCore import QThread, pyqtSignal
import socket, time

class EscuchaServidor(QThread):
    mensaje = pyqtSignal(str)

    def run(self):
        cliente = socket.socket()
        cliente.connect(("localhost", 5000))
        while True:
            data = cliente.recv(1024).decode()
            self.mensaje.emit(data)
```

---